In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("../data/bangkok_traffy.csv")

In [ ]:
df = df.drop(['organization','comment','photo','photo_after','coords','address','star','count_reopen'],axis=1)

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df['province'].unique()

In [ ]:
df = df[df['province'].str.contains('กรุงเทพ|bangkok',na=False,case=False)]

In [ ]:
# df[df['type']=='{}'].time.value_counts()

In [ ]:
df = df.dropna(subset=['subdistrict', 'district'] )

In [ ]:
df.isna().sum()

In [ ]:
df = df[df['state'].str.contains('เสร็จ',na=False,case=False)]

In [ ]:
df.type.unique()

In [ ]:
df.district.value_counts()

In [ ]:
df[['subdistrict','district']]

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)
df['last_activity'] = pd.to_datetime(df['last_activity'], utc=True)

In [ ]:
df['time'] =  (df['last_activity'] - df['timestamp']).dt.days

In [ ]:
type_con = df['type'].str.strip('{}').str.get_dummies(',')

In [ ]:
df = pd.concat([df,type_con],axis=1)

In [ ]:
district_hot =  pd.get_dummies(df['district'])

In [ ]:
df = pd.concat([df,district_hot],axis=1)

In [ ]:
df = df.drop(['ticket_id','type','subdistrict','district','province','timestamp','state','last_activity'],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_rem = train_test_split(df, test_size=0.3, random_state=42)

df_val, df_test = train_test_split(df_rem, test_size=0.5, random_state=42)


# Model

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tqdm import tqdm

class TabularDataset(Dataset):
    def __init__(self, dataframe, feature_scaler, target_scaler):
        self.data = dataframe.to_numpy()
        self.feature_scaler = feature_scaler
        self.target_scaler = target_scaler
        self.targets = self.target_scaler.transform(self.data[:, 0].reshape(-1, 1))
        self.features = self.feature_scaler.transform(self.data[:, 1:])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = torch.tensor(self.features[idx], dtype=torch.float32)
        y = torch.tensor(self.targets[idx], dtype=torch.float32)
        return x, y

In [ ]:
feature_scaler = MinMaxScaler()
feature_scaler.fit(df_train.iloc[:, 1:])

target_scaler = MinMaxScaler()
target_scaler.fit(df_train.iloc[:, 0].values.reshape(-1, 1))

In [ ]:
train_dataset = TabularDataset(df_train, feature_scaler, target_scaler)
val_dataset = TabularDataset(df_val, feature_scaler, target_scaler)
test_dataset = TabularDataset(df_test, feature_scaler, target_scaler)


train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class TabularModel(torch.nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.fc1 = torch.nn.Linear(input_size, 256)
        self.fc2 = torch.nn.Linear(256, 128)
        self.fc3 = torch.nn.Linear(128, 1)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [ ]:
train_dataset[0]

In [ ]:
model = TabularModel(input_size=df_train.shape[1]-1)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
import matplotlib.pyplot as plt

EPOCHS = 5
train_losses = []
test_losses = []

for epoch in range(EPOCHS):
    train_loss = 0.0
    for (inputs, targets) in tqdm(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)
    train_losses.append(train_loss)

    test_loss = 0.0
    for (inputs, targets) in tqdm(val_loader):
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()
    test_loss /= len(test_loader)
    test_losses.append(test_loss)
    
    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.6f}, Test Loss: {test_loss:.6f}')

In [ ]:
# Plotting the loss
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

import pandas as pd

def evaluate_model(model, dataloader, feature_scaler, target_scaler):
    criterion = torch.nn.MSELoss()
    model.eval()
    total_loss = 0.0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, targets in dataloader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
            y_true.extend(targets.numpy())
            y_pred.extend(outputs.numpy())
    
    # Inverse transform the scaled values
    y_true = target_scaler.inverse_transform(y_true)
    y_pred = target_scaler.inverse_transform(y_pred)
    
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    
    df = pd.DataFrame({'Actual': y_true.flatten(), 'Predicted': y_pred.flatten()})
    
    return mse, mae, rmse, df

In [ ]:
train_mse, train_mae, train_rmse, results_df = evaluate_model(model, train_loader, feature_scaler, target_scaler)
print(f'Train MSE: {train_mse:.4f}')
print(f'Train MAE: {train_mae:.4f}')
print(f'Train RMSE: {train_rmse:.4f}')

results_df.head()

In [ ]:
# Usage:
test_mse, test_mae, test_rmse, results_df = evaluate_model(model, test_loader, feature_scaler, target_scaler)
print(f'Test MSE: {test_mse:.4f}')
print(f'Test MAE: {test_mae:.4f}')
print(f'Test RMSE: {test_rmse:.4f}')

results_df.head()  # Print the first few rows of the results DataFrame

In [ ]:
results_df.describe()